In [ ]:
!python -m pip install git+https://github.com/NVIDIA/NeMo.git@r1.20.0#egg=nemo_toolkit[all]


DEPRECATION: git+https://github.com/NVIDIA/NeMo.git@r1.20.0#egg=nemo_toolkit[all] contains an egg fragment with a non-PEP 508 name pip 25.0 will enforce this behaviour change. A possible replacement is to use the req @ url syntax, and remove the egg fragment. Discussion can be found at https://github.com/pypa/pip/issues/11617
  Cloning https://github.com/NVIDIA/NeMo.git (to revision r1.20.0) to /tmp/pip-install-9ygvnd0o/nemo-toolkit_97ecf78f31124496833ce0a002f0f20b
  Running command git clone --filter=blob:none --quiet https://github.com/NVIDIA/NeMo.git /tmp/pip-install-9ygvnd0o/nemo-toolkit_97ecf78f31124496833ce0a002f0f20b
  Running command git checkout -b r1.20.0 --track origin/r1.20.0
  Switched to a new branch 'r1.20.0'
  Branch 'r1.20.0' set up to track remote branch 'r1.20.0' from 'origin'.
  Resolved https://github.com/NVIDIA/NeMo.git to commit 2baef811f21372c3340dd2d82635d2377e78a660
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Prepa

## Clean out csv files and convert to tsv

In [ ]:
import pandas as pd

nabulime = pd.read_csv('/content/drive/MyDrive/CGIAR/Speech Recognition/Data/Raw/pilot/Luganda/transcriptions/Nabulime Madrine - Sheet1.csv')
Nakirijja = pd.read_csv('/content/drive/MyDrive/CGIAR/Speech Recognition/Data/Raw/pilot/Luganda/transcriptions/Nakirijja Faustar - Sheet1.csv')
Nalubega = pd.read_csv('/content/drive/MyDrive/CGIAR/Speech Recognition/Data/Raw/pilot/Luganda/transcriptions/Nalubega Flavia - Sheet1.csv', header=4)
makerere = pd.read_csv('/content/drive/MyDrive/CGIAR/Speech Recognition/Data/Raw/pilot/Luganda/test_samples.csv')

In [ ]:
makerere['path'] = makerere['path'].str.split('_').str[-1]

In [ ]:
nabulime['path'] = nabulime['Audio description'].str[6:]
nabulime.loc[nabulime['path']=='5fa7d98985e219.00823371.wav', 'path'] = '5fb7d98985e219.00823371.wav'
nabulime['Sentence'] = nabulime['Sentence'].str.lower()


In [ ]:
nabulime = nabulime.merge(makerere[['path', 'sentence']], on='path')
nabulime.rename(columns={'sentence':'ground_truth','Sentence':'sentence'}, inplace=True)


In [ ]:
Nalubega['path'] = Nalubega['Audio description'].str[6:-3].str.strip()
Nalubega.loc[Nalubega['path']=='5fb7dba2bd9f1.1461', 'path'] = '5fb7dba2bd94f1.14615480'
Nalubega['Sentence'] = Nalubega['Sentence'].str.lower()


In [ ]:
makerere['part_path'] = makerere['path'].str.findall('|'.join(Nalubega['path'].values.tolist())).str.join(',')

In [ ]:
Nalubega = Nalubega.merge(makerere[['part_path', 'path', 'sentence']], left_on='path', right_on='part_path', suffixes=('_x', None))
Nalubega.rename(columns={'sentence':'ground_truth','Sentence':'sentence'}, inplace=True)


In [ ]:

Nakirijja.loc[Nakirijja['Audio name']=='5fa253186015', 'Audio name'] = '5fa253186d5805'
Nakirijja.loc[Nakirijja['Audio name']=='5fa25fr78', 'Audio name'] = '5fa25f47844a55'
Nakirijja.loc[Nakirijja['Audio name']=='5fb5554ac', 'Audio name'] = '5fbb5554ac58e6'
Nakirijja['Transcription'] = Nakirijja['Transcription'].str.lower()

In [ ]:
makerere['part_path2'] = makerere['path'].str.findall('|'.join(Nakirijja['Audio name'].values.tolist())).str.join(',')

In [ ]:
Nakirijja = Nakirijja.merge(makerere[['part_path2', 'path', 'sentence']], left_on='Audio name', right_on='part_path2', suffixes=('_x', None))
Nakirijja.rename(columns={'sentence':'ground_truth','Transcription':'sentence'}, inplace=True)


In [ ]:
nabulime['sentence'] = nabulime['sentence'].str.replace("’", "'").apply(lambda text: re.sub("[^A-Za-z\']+", ' ', str(text)))
Nakirijja['sentence'] = Nakirijja['sentence'].str.replace("’", "'").apply(lambda text: re.sub("[^A-Za-z\']+", ' ', str(text)))
Nalubega['sentence'] = Nalubega['sentence'].str.replace("’", "'").apply(lambda text: re.sub("[^A-Za-z\']+", ' ', str(text)))

In [ ]:
nabulime.drop(columns=['Number', 'Audio description']).to_csv(
    '/content/drive/MyDrive/CGIAR/Speech Recognition/Data/Raw/pilot/Luganda/transcriptions/Nabulime Madrine - Sheet1.tsv',
    sep="\t", index=False)

Nalubega.drop(columns=['No.', 'Audio description', 'path_x', 'part_path']).to_csv(
    '/content/drive/MyDrive/CGIAR/Speech Recognition/Data/Raw/pilot/Luganda/transcriptions/Nalubega Flavia - Sheet1.tsv',
    sep="\t", index=False)

Nakirijja.drop(columns=['Audio number', 'Audio name', 'part_path2']).to_csv(
    '/content/drive/MyDrive/CGIAR/Speech Recognition/Data/Raw/pilot/Luganda/transcriptions/Nakirijja Faustar - Sheet1.tsv',
    sep="\t", index=False)

## Convert tsv files to json manifests

In [ ]:
#convert tsv files to json manifest
!python /content/tsv_to_json.py \
  --tsv='/content/drive/MyDrive/CGIAR/Speech Recognition/Data/Raw/pilot/Luganda/transcriptions/Nalubega Flavia - Sheet1.tsv' \
  --folder='.' \
  --sampling_count=-1

100% 25/25 [00:00<00:00, 20504.03it/s]


In [ ]:
#convert tsv files to json manifest
!python /content/tsv_to_json.py \
  --tsv='/content/drive/MyDrive/CGIAR/Speech Recognition/Data/Raw/pilot/Luganda/transcriptions/Nabulime Madrine - Sheet1.tsv' \
  --folder='.' \
  --sampling_count=-1

100% 25/25 [00:00<00:00, 20657.53it/s]


In [ ]:
#convert tsv files to json manifest
!python /content/tsv_to_json.py \
  --tsv='/content/drive/MyDrive/CGIAR/Speech Recognition/Data/Raw/pilot/Luganda/transcriptions/Nakirijja Faustar - Sheet1.tsv' \
  --folder='.' \
  --sampling_count=-1

100% 25/25 [00:00<00:00, 23157.60it/s]


## Evaluate transcriptions on WER and CER

In [ ]:
import os

#download eval script
if not os.path.exists("scripts/speech_to_text_eval.py"):
  !wget -P scripts/ https://raw.githubusercontent.com/NVIDIA/NeMo/v1.20.0/examples/asr/speech_to_text_eval.py

--2024-11-18 10:35:07--  https://raw.githubusercontent.com/NVIDIA/NeMo/v1.20.0/examples/asr/speech_to_text_eval.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7084 (6.9K) [text/plain]
Saving to: ‘scripts/speech_to_text_eval.py’

speech_to_text_eval 100%[===================>]   6.92K  --.-KB/s    in 0s      

2024-11-18 10:35:07 (44.5 MB/s) - ‘scripts/speech_to_text_eval.py’ saved [7084/7084]



In [ ]:

!wget -P scripts/ https://raw.githubusercontent.com/NVIDIA/NeMo/v1.20.0/examples/asr/transcribe_speech.py
!wget -P scripts/ https://raw.githubusercontent.com/NVIDIA/NeMo/v1.20.0/examples/asr/asr_chunked_inference/rnnt/speech_to_text_buffered_infer_rnnt.py
!wget -P scripts/ https://raw.githubusercontent.com/NVIDIA/NeMo/v1.20.0/examples/asr/asr_chunked_inference/ctc/speech_to_text_buffered_infer_ctc.py

--2024-11-18 09:01:56--  https://raw.githubusercontent.com/NVIDIA/NeMo/v1.20.0/examples/asr/transcribe_speech.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16580 (16K) [text/plain]
Saving to: ‘scripts/transcribe_speech.py’

transcribe_speech.p 100%[===================>]  16.19K  --.-KB/s    in 0.001s  

2024-11-18 09:01:56 (26.5 MB/s) - ‘scripts/transcribe_speech.py’ saved [16580/16580]

--2024-11-18 09:01:56--  https://raw.githubusercontent.com/NVIDIA/NeMo/v1.20.0/examples/asr/asr_chunked_inference/rnnt/speech_to_text_buffered_infer_rnnt.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... c

In [ ]:
# Calculate WER
!python /content/scripts/speech_to_text_eval.py \
  dataset_manifest=/content/drive/MyDrive/CGIAR/Speech\ Recognition/Data/Raw/pilot/Luganda/transcriptions/Nabulime\ Madrine\ -\ Sheet1.json \
  use_cer=False \
  only_score_manifest=True

# Calculate CER
# !python /content/scripts/speech_to_text_eval.py \
#   dataset_manifest=/content/drive/MyDrive/CGIAR/Speech Recognition/Data/Raw/pilot/Luganda/transcriptions/Nabulime Madrine - Sheet1.json \
#   use_cer=True \
#   only_score_manifest=True

[NeMo W 2024-11-18 11:51:12 nemo_logging:349] /usr/local/lib/python3.10/dist-packages/megatron/core/tensor_parallel/layers.py:220: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
      def forward(ctx, input, weight, bias, gradient_accumulation_fusion,
    
[NeMo W 2024-11-18 11:51:12 nemo_logging:349] /usr/local/lib/python3.10/dist-packages/megatron/core/tensor_parallel/layers.py:250: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
      def backward(ctx, grad_output):
    
[NeMo W 2024-11-18 11:51:18 nemo_logging:349] /usr/local/lib/python3.10/dist-packages/hydra/_internal/hydra.py:119: UserWarning: Future Hydra versions will no longer change working directory at job runtime by default.
    See https://hydra.cc/docs/next/upgrades/1.1_to_1.2/changes_to_job_working_dir/ for more information.
      ret = run

In [ ]:
# Calculate WER
!python /content/scripts/speech_to_text_eval.py \
  dataset_manifest=/content/drive/MyDrive/CGIAR/Speech\ Recognition/Data/Raw/pilot/Luganda/transcriptions/Nakirijja\ Faustar\ -\ Sheet1.json \
  use_cer=False \
  only_score_manifest=True

[NeMo W 2024-11-18 11:51:27 nemo_logging:349] /usr/local/lib/python3.10/dist-packages/megatron/core/tensor_parallel/layers.py:220: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
      def forward(ctx, input, weight, bias, gradient_accumulation_fusion,
    
[NeMo W 2024-11-18 11:51:27 nemo_logging:349] /usr/local/lib/python3.10/dist-packages/megatron/core/tensor_parallel/layers.py:250: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
      def backward(ctx, grad_output):
    
[NeMo W 2024-11-18 11:51:33 nemo_logging:349] /usr/local/lib/python3.10/dist-packages/hydra/_internal/hydra.py:119: UserWarning: Future Hydra versions will no longer change working directory at job runtime by default.
    See https://hydra.cc/docs/next/upgrades/1.1_to_1.2/changes_to_job_working_dir/ for more information.
      ret = run

In [ ]:
# Calculate WER
!python /content/scripts/speech_to_text_eval.py \
  dataset_manifest=/content/drive/MyDrive/CGIAR/Speech\ Recognition/Data/Raw/pilot/Luganda/transcriptions/Nalubega\ Flavia\ -\ Sheet1.json \
  use_cer=False \
  only_score_manifest=True

[NeMo W 2024-11-18 11:51:41 nemo_logging:349] /usr/local/lib/python3.10/dist-packages/megatron/core/tensor_parallel/layers.py:220: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
      def forward(ctx, input, weight, bias, gradient_accumulation_fusion,
    
[NeMo W 2024-11-18 11:51:41 nemo_logging:349] /usr/local/lib/python3.10/dist-packages/megatron/core/tensor_parallel/layers.py:250: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
      def backward(ctx, grad_output):
    
[NeMo W 2024-11-18 11:51:47 nemo_logging:349] /usr/local/lib/python3.10/dist-packages/hydra/_internal/hydra.py:119: UserWarning: Future Hydra versions will no longer change working directory at job runtime by default.
    See https://hydra.cc/docs/next/upgrades/1.1_to_1.2/changes_to_job_working_dir/ for more information.
      ret = run

In [ ]:
nabulime_processed = pd.read_csv('/content/drive/MyDrive/CGIAR/Speech Recognition/Data/Raw/pilot/Luganda/transcriptions/Nabulime Madrine - Sheet1.tsv', sep='\t')
nakirijja_processed = pd.read_csv('/content/drive/MyDrive/CGIAR/Speech Recognition/Data/Raw/pilot/Luganda/transcriptions/Nakirijja Faustar - Sheet1.tsv', sep='\t')
nalubega_processed = pd.read_csv('/content/drive/MyDrive/CGIAR/Speech Recognition/Data/Raw/pilot/Luganda/transcriptions/Nalubega Flavia - Sheet1.tsv', sep='\t')

In [ ]:
def WER_interpret(data):
    df = data.copy()

    df['word_substitutions'] = [[(r,h)
    for r,h in zip(ref.split(), hyp.split())
    if r != h]
    for ref, hyp in df[['ground_truth', 'sentence']].apply(tuple, axis=1) ]

    df['deleted_words'] = [set(ref.split())-set(hyp.split())
    for ref, hyp in df[['ground_truth', 'sentence']].apply(tuple, axis=1)]

    df['inserted_words'] = [set(hyp.split())-set(ref.split())
    for ref, hyp in df[['ground_truth', 'sentence']].apply(tuple, axis=1)]

    df['lev_dist'] = [editdistance.eval(hyp.split(), ref.split())
    for ref, hyp in df[['ground_truth', 'sentence']].apply(tuple, axis=1)]


    df['deletions'] = df['ground_truth'].str.split(' ').str.len() - df['sentence'].str.split(' ').str.len()
    df['insertions'] = df['sentence'].str.split(' ').str.len() - df['ground_truth'].str.split(' ').str.len()
    df['substitutions'] = df['word_substitutions'].str.len()
    # df['WER'] = ((df['substitutions']+df['insertions']+df['deletions'])/df['ground_truth'].str.split(' ').str.len())*100
    df['WER_lev'] = (df['lev_dist']/df['ground_truth'].str.split(' ').str.len())*100

    return df

In [ ]:
nabulime_interpret = WER_interpret(nabulime_processed)
nakirijja_interpret = WER_interpret(nakirijja_processed)
nalubega_interpret = WER_interpret(nalubega_processed)

In [ ]:
nakirijja_interpret

,sentence,path,ground_truth,word_substitutions,deleted_words,inserted_words,lev_dist,deletions,insertions,substitutions,WER_lev
0,babeera n'obusero obwo nga babwesibyemu mu kiw...,5fa16d12759d42.84890669.wav,babeera n'obusero obwo nga babwesibyeemu mu ki...,"[(babwesibyeemu, babwesibyemu), (ng'akesibyeem...","{ko, babwesibyeemu, ng'akesibyeemu}","{ng'akeesibyemukko, babwesibyemu}",3,0,0,2,25.000000
1,um omulungi awuliriza tulina omuzannyo gwaffe...,5fa25f47844a55.18939112.wav,um omulungi awuliriza tiyina omuzannyo gwaffe ...,"[(tiyina, tulina), (gwetutambulirako, gwe), (e...","{tiyina, tuyina, gwetutambulirako}","{tulina, gwe, tutambulirako}",4,-3,3,16,19.047619
2,a oba ngenda kuggyamu muddo b ye ssebo a omuli...,5fa165ce5eb952.24851209.wav,oba ng'enda um kujjamu muddo um sebo omirimi y...,"[(oba, a), (ng'enda, oba), (um, ngenda), (kujj...","{sebo, omirimi, kujjamu, um, yalibadde, ng'enda}","{kuggyamu, a, b, ssebo, ngenda, omulimi, yandi...",10,-3,3,14,66.666667
3,ebibuuzo we biggyira wonna ebyetaaga okuba nga...,5fa255c13c2b36.94511888.wav,ebibuuzo webiggira wonna ebyetaaga okuba nga b...,"[(webiggira, we), (wonna, biggyira), (ebyetaag...",{webiggira},"{we, biggyira}",2,-2,2,7,25.000000
4,ng'enjogera ya leero bwegamba ayinza n'okuyita...,5fa263b7819ff3.28268026.wav,nga enjogera ya leero bw'egamba ayinza n'okuyi...,"[(nga, ng'enjogera), (enjogera, ya), (ya, leer...","{kattaka, nga, bw'egamba, gula, enjogera}","{ng'enjogera, bwegamba, kataka, ggula}",5,0,0,12,38.461538
5,a um kaakati um njagala tutusinziire ku kimu k...,5fa2603f71bdf8.57311791.wav,um kaakati um njagala tu tusinzire ku kimu ku ...,"[(um, a), (kaakati, um), (um, kaakati), (njaga...","{sebo, tu, tutunulire, tusinzire}","{a, b, ssebo, tutunuulire, tutusinziire}",7,-3,3,10,50.000000
6,era n'oluvannyuma ne tukubuuza ekikula kyo oli...,5fa2646b0b2029.14288016.wav,era n'oluvannyuma netukubuuza ekikula kyo oli ...,"[(netukubuuza, ne), (ekikula, tukubuuza), (kyo...","{ojjakunyiga, um, netukubuuza, musa}","{kunyiga, ojja, tukubuuza, musaa, ne}",8,-3,3,19,38.095238
7,a abalimi balukazaako kitiko b ye ssebo a ffe ...,5fa16673d50899.94095718.wav,abalimi babukazaako kitiko ye sebo ffe mulu mu...,"[(abalimi, a), (babukazaako, abalimi), (kitiko...","{sebo, olwobulimi, babukazaako, mulu, mululimi...","{mu, lu, olw'obulimi, balukazaako, a, b, lulim...",12,-6,6,14,85.714286
8,noolwekyo ebintu bino ebikwatagana n'ebbanja t...,5fa17734d5faf7.10600131.wav,n'olwekyo um ebintu bino ebikwatagana n'ebbanj...,"[(n'olwekyo, noolwekyo), (um, ebintu), (ebintu...","{lwaki, um, n'olwekyo}",{noolwekyo},3,1,-1,8,30.000000
9,a wakati wakati w'okaluba n'okugonda b okay a ...,5fa253186d5805.05414241.wav,wakati wakati w'okukaluba n'okugonda okay fift...,"[(wakati, a), (w'okukaluba, wakati), (n'okugon...","{w'okukaluba, okute}","{a, w'okaluba, b}",5,-3,3,13,35.714286


In [ ]:
nabulime_processed['deleted_words'] = [set(ref.split())-set(hyp.split())
for ref, hyp in nabulime_processed[['ground_truth', 'sentence']].apply(tuple, axis=1)]

In [ ]:
nabulime_processed['inserted_words'] = [set(hyp.split())-set(ref.split())
for ref, hyp in nabulime_processed[['ground_truth', 'sentence']].apply(tuple, axis=1)]

In [ ]:
nabulime_processed['lev_dist'] = [editdistance.eval(hyp.split(), ref.split())
for ref, hyp in nabulime_processed[['ground_truth', 'sentence']].apply(tuple, axis=1)]

In [ ]:
import editdistance

nabulime_processed['deletions'] = nabulime_processed['ground_truth'].str.split(' ').str.len() - nabulime_processed['sentence'].str.split(' ').str.len()
nabulime_processed['insertions'] = nabulime_processed['sentence'].str.split(' ').str.len() - nabulime_processed['ground_truth'].str.split(' ').str.len()
nabulime_processed['substitutions'] = nabulime_processed['word_substitutions'].str.len()
# nabulime_processed['WER'] = ((nabulime_processed['substitutions']+nabulime_processed['insertions']+nabulime_processed['deletions'])/nabulime_processed['ground_truth'].str.split(' ').str.len())*100
nabulime_processed['WER_lev'] = (nabulime_processed['lev_dist']/nabulime_processed['ground_truth'].str.split(' ').str.len())*100

In [ ]:
nakirijja_interpret['WER'].mean()

58.06598703938857

In [ ]:
nalubega_interpret['WER_lev'].mean()


39.14816947758124

In [ ]:
OUTPUT_DIR = '/content/drive/MyDrive/CGIAR/Speech Recognition/Data/Raw/pilot/Luganda/transcripts_eval'

nabulime_interpret.to_csv(f'{OUTPUT_DIR}/Nabulime Madrine.csv', index=False)
nakirijja_interpret.to_csv(f'{OUTPUT_DIR}/Nakirijja Faustar.csv', index=False)
nalubega_interpret.to_csv(f'{OUTPUT_DIR}/Nalubega Flavia.csv', index=False)

## Clean out punctuations

In [ ]:
import re
import string

hyp_punct = [(set([punct for w in word for punct in re.findall(r'[^a-zA-Z]', w)])) for word in nakirijja_processed['sentence'].str.split()]
ref_punct = [(set([punct for w in word for punct in re.findall(r'[^a-zA-Z]', w)])) for word in nakirijja_processed['ground_truth'].str.split()]

In [ ]:
nabulime_processed['sentence'] = nabulime_processed['sentence'].str.replace("’", "'").apply(lambda text: re.sub("[^A-Za-z\']+", ' ', str(text)))
nakirijja_processed['sentence'] = nakirijja_processed['sentence'].str.replace("’", "'").apply(lambda text: re.sub("[^A-Za-z\']+", ' ', str(text)))
nalubega_processed['sentence'] = nalubega_processed['sentence'].str.replace("’", "'").apply(lambda text: re.sub("[^A-Za-z\']+", ' ', str(text)))

In [ ]:
ref_punct

[{"'"},
 {"'"},
 {"'"},
 set(),
 {"'"},
 set(),
 {"'"},
 set(),
 {"'"},
 {"'"},
 set(),
 set(),
 set(),
 {"'"},
 {"'"},
 {"'"},
 set(),
 set(),
 {"'"},
 {"'"},
 set(),
 set(),
 {"'"},
 set(),
 {"'"}]

In [ ]:
hyp_punct

[{"'"},
 {"'"},
 set(),
 set(),
 {"'"},
 set(),
 {"'"},
 {"'"},
 {"'"},
 {"'"},
 {"'"},
 set(),
 set(),
 {"'"},
 {"'"},
 {"'"},
 set(),
 set(),
 {"'"},
 set(),
 set(),
 set(),
 {"'"},
 set(),
 set()]

In [ ]:
nabulime_processed

,sentence,path,ground_truth,word_substitutions,deleted_words,inserted_words
0,babeera n’obusero obwo nga babwesibyemu mu kiw...,5fa16d12759d42.84890669.wav,babeera n'obusero obwo nga babwesibyeemu mu ki...,"[(n’obusero, n'obusero), (babwesibyemu, babwes...","{ng’akeesibyemukko., n’obusero, babwesibyemu}","{ko, n'obusero, babwesibyeemu, ng'akesibyeemu}"
1,"um, omulungi awuliriza tuyina omuzannyo gwaffe...",5fa25f47844a55.18939112.wav,um omulungi awuliriza tiyina omuzannyo gwaffe ...,"[(um,, um), (tuyina, tiyina), (gwe, gwetutambu...","{n’omulamwa., musoke., tutambulirako, guno., g...","{guno, musoke, um, tiyina, n'omulamwa, gwetuta..."
2,"oba ŋenda kuggyamu muddo, um ssebo, omulimi ya...",5fa165ce5eb952.24851209.wav,oba ng'enda um kujjamu muddo um sebo omirimi y...,"[(ŋenda, ng'enda), (kuggyamu, um), (muddo,, ku...","{kuggyamu, omulimi, ssebo,, muddo,, ŋenda}","{sebo, omirimi, muddo, kujjamu, ng'enda}"
3,bibuuzo webijjira wonna ebyetaaga okuba nga by...,5fa255c13c2b36.94511888.wav,ebibuuzo webiggira wonna ebyetaaga okuba nga b...,"[(bibuuzo, ebibuuzo), (webijjira, webiggira), ...","{abakugu.., bibuuzo, webijjira}","{abakugu, webiggira, ebibuuzo}"
4,ng’enjogera ya leero bwegamba ayinza n’okkuyit...,5fa263b7819ff3.28268026.wav,nga enjogera ya leero bw'egamba ayinza n'okuyi...,"[(ng’enjogera, nga), (ya, enjogera), (leero, y...","{bwegamba, n’okkuyitamu, ng’enjogera, kataka.,...","{kattaka, nga, n'okuyitamu, bw'egamba, gula, e..."
5,"um, kaakati aa njagala tusinziire ku kimu ku k...",5fa2603f71bdf8.57311791.wav,um kaakati um njagala tu tusinzire ku kimu ku ...,"[(um,, um), (aa, um), (tusinziire, tu), (ku, t...","{aa, ekigimusa., tusinziire, um,, ssebo,, tutu...","{sebo, tutunulire, tusinzire, tu, ekigimusa, ye}"
6,"era n’oluvannyuma ne tukubuuza ekikula kyo, ol...",5fa2646b0b2029.14288016.wav,era n'oluvannyuma netukubuuza ekikula kyo oli ...,"[(n’oluvannyuma, n'oluvannyuma), (ne, netukubu...","{kyo,, ne, kunyiga, bw’oba, ojja, mukyala,, n’...","{bbiri, um, netukubuuza, n'oluvannyuma, bwoba,..."
7,"obulimi baabukazaako kitiko, um ssebo, ffe mu ...",5fa16673d50899.94095718.wav,abalimi babukazaako kitiko ye sebo ffe mulu mu...,"[(obulimi, abalimi), (baabukazaako, babukazaak...","{mu, kitiko,, olw’obulimi, baabukazaako, um, l...","{sebo, olwobulimi, babukazaako, kitiko, mulu, ..."
8,"noolwekyo, ebintu bino ebikwatagana ne bbanja,...",5fa17734d5faf7.10600131.wav,n'olwekyo um ebintu bino ebikwatagana n'ebbanj...,"[(noolwekyo,, n'olwekyo), (ebintu, um), (bino,...","{nti., noolwekyo,, kyekyo, bbanja,, ne}","{um, nti, n'ebbanja, n'olwekyo}"
9,"wakati wakati wo kkaluba n’okugonda, fifty fif...",5fa253186d5805.05414241.wav,wakati wakati w'okukaluba n'okugonda okay fift...,"[(wo, w'okukaluba), (kkaluba, n'okugonda), (n’...","{fifty,, wo, kkaluba, ki?, n’okugonda,, okute,}","{w'okukaluba, okay, n'okugonda, ki, okute}"
